In [11]:
import datetime

import csv

import pandas as pd

from sqlalchemy import create_engine

import yfinance as yf 

from bs4 import BeautifulSoup

import requests

from pyspark.sql import SparkSession

import pyspark

from pyspark.context import SparkContext

import pyspark.sql.functions as fc

import pymysql

from pyspark.sql import Row

from pyspark.sql.types import *

In [2]:
sc = SparkContext.getOrCreate()

spark = SparkSession(sc)

In [5]:
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='PApOEaPbJNOM62p1',
                             db='stockdb'
                            ,port=1234)

In [6]:
cur = connection.cursor()



In [3]:
def industrycompany(cursor,Industry):
    sqlcode=('Select * From CompanyInformation Where Sector="'+Industry+'"')
    cursor.execute(sqlcode)
    Industry = cursor.fetchall()
    compname=[]
    for i in Industry:
        compname.append(i[0])
        
    return compname

In [7]:
compname=industrycompany(cur,'Industrials')

In [8]:
def getprice(name):
    #newdict=dict()
    newlist=[]
    for i in name:
        newdict=dict()
        url='https://finance.yahoo.com/quote/'
        finalurl=url+i+'?p='+i+'&.tsrc=fin-srch'
        #print(finalurl)
        r=requests.get(finalurl)
        soup = BeautifulSoup(r.content,'lxml')
        stockprice=soup.find_all('div',{"My(6px) Pos(r) smartphone_Mt(6px)"})[0].find('span').text
        #print(stockprice)
        newdict['name']=i
        newdict['price']=stockprice
        newlist.append(newdict)
    return newlist

In [9]:
compdict=getprice(compname)

In [12]:
rdd = sc.parallelize(compdict)
schema = StructType([ StructField("name", StringType(), True),StructField("price", StringType(), True)])

In [13]:
df3 = spark.createDataFrame(rdd, schema)

In [14]:
df3.collect()

[Row(name='AAL', price='12.04'),
 Row(name='ALK', price='44.34'),
 Row(name='ALLE', price='112.36'),
 Row(name='AME', price='116.76'),
 Row(name='AOS', price='54.82'),
 Row(name='BA', price='182.15'),
 Row(name='CARR', price='38.06'),
 Row(name='CAT', price='169.46'),
 Row(name='CHRW', price='93.33'),
 Row(name='CMI', price='234.90'),
 Row(name='CPRT', price='117.80'),
 Row(name='CSX', price='90.04'),
 Row(name='CTAS', price='360.04'),
 Row(name='DAL', price='35.02'),
 Row(name='DE', price='249.35'),
 Row(name='DOV', price='120.87'),
 Row(name='EFX', price='159.19'),
 Row(name='EMR', price='74.90'),
 Row(name='ETN', price='114.07'),
 Row(name='EXPD', price='88.61'),
 Row(name='FAST', price='47.24'),
 Row(name='FBHS', price='83.05'),
 Row(name='FDX', price='267.81'),
 Row(name='FLS', price='31.93'),
 Row(name='FTV', price='70.00'),
 Row(name='GD', price='150.29'),
 Row(name='GE', price='8.89'),
 Row(name='GWW', price='403.39'),
 Row(name='HII', price='158.04'),
 Row(name='HON', price='1